# Setup

In [ ]:
import os
import logging as log

from lisa.utils import setup_logging
setup_logging(level=log.ERROR)

from lisa.trace import Trace
from lisa.wa import WAOutput
from lisa.stats import Stats
from lisa.datautils import series_mean
from pandas import DataFrame
import pandas as pd
import scipy as sp
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import holoviews as hv
from holoviews import opts
from bokeh.themes import built_in_themes
from tabulate import tabulate

from holoviews.operation.datashader import datashade, rasterize
from holoviews.operation import decimate

from wp.notebook import WorkloadNotebookAnalysis, setup_notebook

setup_notebook()

## Runs

In [ ]:
gb = WorkloadNotebookAnalysis('/home/kajpuc01/power/pixel6/geekbench/', [
    'geekbench_product_baseline_3_1407',
    'geekbench_product_arm_vh_sched_debug_3_1907',
], label='Geekbench 6')

gb.show()

# Benchmark scores

## Line plot

In [ ]:
gb.plot.results_line(['score', 'multicore_score'])

## Bar plot

In [ ]:
gb.plot.results_bar(['score', 'multicore_score'])

# Overutilized

## Line plot

In [ ]:
gb.plot.overutilized_line()

# Perf

## Line plot

In [ ]:
gb.plot.perf_line()

## Bar plot

In [ ]:
gb.plot.perf_bar()

# Idle residency

## Bar plot

In [ ]:
gb.plot.idle_residency_bar()

# Idle misses

## Bar plot

In [ ]:
gb.plot.idle_miss_bar()

# Power usage

## Line plot

In [ ]:
gb.plot.power_meter_line()

## Bar plot

In [ ]:
gb.plot.power_meter_bar()

# Energy estimate

## Line plot

In [ ]:
gb.plot.energy_estimate_line()

## Bar plot

In [ ]:
gb.plot.energy_estimate_bar()

# Thermal

## Line plot

In [ ]:
gb.plot.thermal_line()

## Bar plot

In [ ]:
gb.plot.thermal_bar()

# Frequency

## Line plot

In [ ]:
gb.plot.frequency_line()

## Bar plot

In [ ]:
gb.plot.frequency_bar()

# CFS signals

## Line plot

In [ ]:
gb.plot.sched_pelt_cfs_line()

## Bar plot

In [ ]:
gb.plot.sched_pelt_cfs_bar()

# Task wakeup latency

## Line plot

In [ ]:
gb.plot.wakeup_latency_line()

## Bar plot

In [ ]:
gb.plot.wakeup_latency_bar()

## Quantiles

In [ ]:
gb.plot.wakeup_latency_quantiles_bar()

## Per-execution cluster

In [ ]:
gb.plot.wakeup_latency_execution_cluster_bar()

## Per-target cluster

In [ ]:
gb.plot.wakeup_latency_target_cluster_bar()

# Wakeup latency - cgroups

## Line plot

In [ ]:
gb.plot.wakeup_latency_cgroup_line()

## Bar plot

In [ ]:
gb.plot.wakeup_latency_cgroup_bar()

## Quantiles

In [ ]:
gb.plot.wakeup_latency_cgroup_quantiles_bar()

# Task CPU residency

## Clusters - Line plot

In [ ]:
gb.plot.tasks_cpu_residency_cluster_line()

## Clusters - Bar plot

In [ ]:
gb.plot.tasks_cpu_residency_cluster_bar()

## Clusters - Per-task bar plot

In [ ]:
gb.plot.tasks_cpu_residency_per_task_bar()

## CPUs - Bar plot

In [ ]:
gb.plot.tasks_cpu_residency_cpu_bar()

# CPU residency - cgroups

## Clusters - Bar plot

In [ ]:
gb.plot.cgroup_cpu_residency_cluster_bar()

## CPUs - Bar plot

In [ ]:
gb.plot.cgroup_cpu_residency_cpu_bar()

# Summary - TLDR

In [ ]:
gb.plot.summary()